In [2]:

import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib
import mplfinance as mpf
import talib

In [3]:

def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time

_start_time, _end_time = now(3)

data = requests.get(f"https://ftx.com/api//markets/{'BTC-PERP'}/candles?resolution={3600}&start_time={_start_time}&end_time={_end_time}").json()

data = pd.DataFrame(data['result'])

data['time'] = pd.to_datetime(data['time'], unit='ms')
data.set_index('time', inplace= True)
#print(data.head(5))

In [4]:
def getmarkets(PERP):
    markets = requests.get(f"https://ftx.com/api/markets").json()
    markets = pd.DataFrame(markets['result'])
    markets = markets['name'].tolist()
    markets = [market for market in markets if market.endswith(PERP)]
    return markets

perpnames = getmarkets('PERP')
print(perpnames)

['1INCH-PERP', 'AAVE-PERP', 'ADA-PERP', 'AGLD-PERP', 'ALCX-PERP', 'ALGO-PERP', 'ALICE-PERP', 'ALPHA-PERP', 'ALT-PERP', 'AMPL-PERP', 'AR-PERP', 'ASD-PERP', 'ATLAS-PERP', 'ATOM-PERP', 'AUDIO-PERP', 'AVAX-PERP', 'AXS-PERP', 'BADGER-PERP', 'BAL-PERP', 'BAND-PERP', 'BAO-PERP', 'BAT-PERP', 'BCH-PERP', 'BIT-PERP', 'BNB-PERP', 'BNT-PERP', 'BOBA-PERP', 'BRZ-PERP', 'BSV-PERP', 'BTC-PERP', 'BTT-PERP', 'C98-PERP', 'CAKE-PERP', 'CEL-PERP', 'CELO-PERP', 'CHR-PERP', 'CHZ-PERP', 'CLV-PERP', 'COMP-PERP', 'CONV-PERP', 'CREAM-PERP', 'CRO-PERP', 'CRV-PERP', 'CUSDT-PERP', 'CVC-PERP', 'DASH-PERP', 'DAWN-PERP', 'DEFI-PERP', 'DENT-PERP', 'DODO-PERP', 'DOGE-PERP', 'DOT-PERP', 'DRGN-PERP', 'DYDX-PERP', 'EDEN-PERP', 'EGLD-PERP', 'ENJ-PERP', 'ENS-PERP', 'EOS-PERP', 'ETC-PERP', 'ETH-PERP', 'EXCH-PERP', 'FIDA-PERP', 'FIL-PERP', 'FLM-PERP', 'FLOW-PERP', 'FTM-PERP', 'FTT-PERP', 'GALA-PERP', 'GRT-PERP', 'HBAR-PERP', 'HNT-PERP', 'HOLY-PERP', 'HOT-PERP', 'HT-PERP', 'HUM-PERP', 'ICP-PERP', 'ICX-PERP', 'IMX-PERP', 'IOTA-P

In [5]:
def ohlc(perpnames, tf, daysback):
    _start_time, _end_time = now(daysback) 
    data = requests.get(f"https://ftx.com/api//markets/{perpnames}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()
    data = pd.DataFrame(data['result'])
    return data

In [6]:
dfs = []
tf = 300
daysback = 2
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, daysback))

In [7]:
all = pd.concat(dict(zip(perpnames,dfs)), axis=1)
#closes = all.loc[:,all.columns.get_level_values(1).isin(['close'])]

nameless = pd.concat(dfs, axis=1)
closes = nameless.loc[:,nameless.columns.get_level_values(0).isin(['close'])]
all

1INCH-PERP                                                \
                     startTime          time    open    high     low   close   
0    2022-03-11T07:15:00+00:00  1.646983e+12  1.3430  1.3430  1.3387  1.3388   
1    2022-03-11T07:20:00+00:00  1.646983e+12  1.3388  1.3421  1.3384  1.3421   
2    2022-03-11T07:25:00+00:00  1.646984e+12  1.3421  1.3489  1.3416  1.3482   
3    2022-03-11T07:30:00+00:00  1.646984e+12  1.3482  1.3486  1.3445  1.3479   
4    2022-03-11T07:35:00+00:00  1.646984e+12  1.3479  1.3479  1.3435  1.3467   
..                         ...           ...     ...     ...     ...     ...   
571  2022-03-13T06:50:00+00:00  1.647154e+12  1.3932  1.3950  1.3917  1.3945   
572  2022-03-13T06:55:00+00:00  1.647154e+12  1.3945  1.3945  1.3884  1.3906   
573  2022-03-13T07:00:00+00:00  1.647155e+12  1.3906  1.3922  1.3876  1.3899   
574  2022-03-13T07:05:00+00:00  1.647155e+12  1.3899  1.3933  1.3897  1.3911   
575  2022-03-13T07:10:00+00:00  1.647155e+12  1.3911  1.3922  1.3905  1.3907   

                                 AAVE-PERP                        ...  \
         volume                  startTime          time    open  ...   
0    17449.7818  2022-03-11T07:15:00+00:00  1.646983e+12  118.25  ...   
1     8888.1745  2022-03-11T07:20:00+00:00  1.646983e+12  118.22  ...   
2    33234.1819  2022-03-11T07:25:00+00:00  1.646984e+12  118.51  ...   
3    20601.5779  2022-03-11T07:30:00+00:00  1.646984e+12  118.87  ...   
4    21483.5451  2022-03-11T07:35:00+00:00  1.646984e+12  118.76  ...   
..          ...                        ...           ...     ...  ...   
571  43673.7174  2022-03-13T06:50:00+00:00  1.647154e+12  119.95  ...   
572  74165.7821  2022-03-13T06:55:00+00:00  1.647154e+12  120.13  ...   
573  81135.1621  2022-03-13T07:00:00+00:00  1.647155e+12  120.04  ...   
574  31554.8131  2022-03-13T07:05:00+00:00  1.647155e+12  120.14  ...   
575   6925.1004  2022-03-13T07:10:00+00:00  1.647155e+12  119.96  ...   

    ZIL-PERP                                        ZRX-PERP                \
         low    close      volume                  startTime          time   
0    0.04115  0.04117  12947.6709  2022-03-11T07:15:00+00:00  1.646983e+12   
1    0.04106  0.04115  20597.7199  2022-03-11T07:20:00+00:00  1.646983e+12   
2    0.04115  0.04128   9629.9892  2022-03-11T07:25:00+00:00  1.646984e+12   
3    0.04123  0.04138  14148.7987  2022-03-11T07:30:00+00:00  1.646984e+12   
4    0.04126  0.04134   6963.5823  2022-03-11T07:35:00+00:00  1.646984e+12   
..       ...      ...         ...                        ...           ...   
571  0.04092  0.04095   5500.8694  2022-03-13T06:50:00+00:00  1.647154e+12   
572  0.04094  0.04099   5303.3497  2022-03-13T06:55:00+00:00  1.647154e+12   
573  0.04099  0.04108   3518.3363  2022-03-13T07:00:00+00:00  1.647155e+12   
574  0.04104  0.04106  11454.4338  2022-03-13T07:05:00+00:00  1.647155e+12   
575  0.04101  0.04104    642.3849  2022-03-13T07:10:00+00:00  1.647155e+12   

                                                     
        open     high      low    close      volume  
0    0.50175  0.50175  0.50100  0.50125   196.56050  
1    0.50125  0.50225  0.50125  0.50225    22.05525  
2    0.50225  0.50450  0.50225  0.50425  4636.83000  
3    0.50425  0.50450  0.50375  0.50450  1816.83775  
4    0.50450  0.50500  0.50425  0.50475  4734.23150  
..       ...      ...      ...      ...         ...  
571  0.50150  0.50225  0.50150  0.50225   257.02400  
572  0.50225  0.50225  0.50200  0.50200   535.26525  
573  0.50200  0.50275  0.50200  0.50250  2288.78050  
574  0.50250  0.50275  0.50250  0.50250  1466.47275  
575  0.50250  0.50250  0.50250  0.50250   778.87500  

[576 rows x 1253 columns]

In [8]:
def divs(closes, columns, ob=70, os=30, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = talib.RSI(closes.iloc[:, columns])
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    bearishrsidiv = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    bullishrsidiv = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
   # closes.insert(1, 'bearish_rsi_div', bearishrsidiv)
   # closes.insert(2, 'bullish_rsi_div', bullishrsidiv)
    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['bullish_rsi_div', 'bearish_rsi_div']]


In [9]:
closes.iloc[-1:,]


,close,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
575,1.3907,119.85,0.79602,0.977,92.7,0.711,5.8875,0.2698,3143.5,0.9479,...,0.09405,0.187175,192.19,0.8005,3.0649,18530.0,1855.25,148.41,0.04104,0.5025


In [44]:
rsidata = [] 
for column in range(179):
   rsidata.append(divs(closes, column))
#alle = pd.concat(dict(zip(perpnames,rsidata)), axis=1)
øve = pd.concat(rsidata, axis = 1)
øve


C:\Users\jonas\AppData\Local\Temp\ipykernel_15152\3791602454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['RSI'] = talib.RSI(closes.iloc[:, columns])
C:\Users\jonas\AppData\Local\Temp\ipykernel_15152\3791602454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
C:\Users\jonas\AppData\Local\Temp\ipykernel_15152\3791602454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,...,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
573,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
øve.iloc[:1] 

,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,...,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
div_df = pd.DataFrame()
def get_divs(øve):
   øve = øve.iloc[-1:]
   div_df['dives'] = np.where(øve.iloc[-1:] == 0, True, False)
   return div_df
get_divs(øve)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series